In [1]:
import os
import redis
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# Redis 연결 정보 읽기
REDIS_HOST = os.getenv('REDIS_HOST')
REDIS_PORT = int(os.getenv('REDIS_PORT'))
REDIS_USERNAME = os.getenv('REDIS_USERNAME')
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# Redis 클라이언트 설정
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    username=REDIS_USERNAME,
    password=REDIS_PASSWORD,
    decode_responses=True
)

In [2]:
import os
import pandas as pd
from collections import Counter
from kiwipiepy import Kiwi
import json

# 데이터셋 경로 설정
data_folder = "../Completed_csv"

# 형태소 분석기 초기화
kiwi = Kiwi()

# 불용어 리스트
custom_stopwords = [
    # 시간 관련
    "지난해", "이날", "오늘", "내일", "올해", "시간", "지난", "이번", "다음", 
    "현재", "그동안", "동안", "오전", "오후", "정오", "새벽", "하루", "주말", 
    "평일", "최근", "과거", "앞으로", "올해", "작년", "내년", "매일", "매주", "매월", 
    "이후", "이전", "이때", "그때", "그전", "그후", "얼마나", "종종", "한때", "언젠가",
    "점심", "저녁", "아침", "밤", "밤새", "낮", "낮시간", "저녁시간", "오후시간",
    
    # 공간 관련
    "대한", "관련", "서울", "한국", "대한민국", "곳", "지역", "전국", "해외", 
    "수도권", "지방", "동네", "거리", "건물", "도시", "국가", "세계", "지구", "근처",
    
    # 단위 및 숫자
    "수", "것", "명", "번", "일", "월", "년", "억", "조", "퍼센트", "위", "분기",
    "만원", "달러", "킬로미터", "미터", "그램", "톤", "리터", "배", "차례", "시간대",
    
    # 조사 및 연결어
    "그리고", "하지만", "그러나", "또는", "또한", "등", "등등", "때문에", "이어서", 
    "뿐만", "이외", "위해", "더욱", "혹은", "따라서", "같이", "그렇지만", "결국", "즉",
    
    # 기타 불용어
    "있다", "없다", "이다", "된다", "한다", "됐다", "중", "대한", "관련", "하기", "하는",
    "부터", "까지", "만큼", "정도", "약", "이상", "이하", "속", "안", "밖", "뿐", "조차",
    "처럼", "과", "또", "더", "모두", "전체", "각", "각각", "모든", "여러", "대부분",
    
    # 주요 불용어 추가
    "최대", "대비", "가능", "공개", "규모", "증가", "감소", "전망", "기준", "수사", 
    "발표", "결정", "확인", "의견", "해결", "논의", "진행", "개최", "지원", "참여",
    "성과", "방안", "조치", "역할", "상황", "현황", "내용", "자료", "정보", "관련", 
    "기대", "효과", "결과", "시작", "종료", "목적", "요청", "확보", "활동", "점검",
    "조사", "문제", "과제", "성과", "변화", "대응", "위기", "차이", "현상",

    # 언론사
    "파이낸셜뉴스", "중앙일보", "서울경제", "아시아경제", "아시아투데이",
    "세계일보", "경향신문", "한국경제", "KBS", "YTN", "국민일보", "한겨레", "아주경제",
    "SBS", "DongAh", "머니투데이",
]

# 데이터베이스 초기화 함수
def reset_redis():
    try:
        print("Initializing Redis database...")
        r.flushdb()  # 현재 Redis 데이터베이스를 비웁니다.
        print("Redis database cleared.")
    except Exception as e:
        print(f"Error clearing Redis database: {e}")
        
# 명사 추출
def extract_nouns(text):
    results = []
    analysis = kiwi.analyze(text)
    for token, pos, _, _ in analysis[0][0]:
        if len(token) > 1 and (pos.startswith('N') or pos.startswith('SL')) and token not in custom_stopwords:
            results.append(token)
    return results

# CSV 파일 로드 및 텍스트 결합
def load_and_combine_text(data_folder):
    combined_texts = []
    all_data = pd.DataFrame()

    for filename in os.listdir(data_folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath, encoding="utf-8")
            if "title" in df.columns and "short_content" in df.columns:
                df["combined"] = df["title"].fillna("") + " " + df["short_content"].fillna("")
                combined_texts.extend(df["combined"].tolist())
                all_data = pd.concat([all_data, df], ignore_index=True)

    return combined_texts, all_data

# 키워드 추출
def extract_top_keywords(texts, top_n=50):
    all_nouns = []
    for text in texts:
        nouns = extract_nouns(text)
        all_nouns.extend(nouns)
    word_counts = Counter(all_nouns)
    return word_counts.most_common(top_n)

# JSON 파일로 데이터 저장
def save_preview_to_json(df, keywords, output_file="preview.json"):
    preview_data = {
        "keywords": [kw for kw, _ in keywords],
        "articles": {
            keyword: df[df["combined"].str.contains(keyword)].to_dict(orient="records")
            for keyword, _ in keywords
        },
        "company_stats": dict(Counter(df["company"])),
        "keyword_stats": {
            keyword: len(df[df["combined"].str.contains(keyword)]) for keyword, _ in keywords
        },
    }
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(preview_data, f, ensure_ascii=False, indent=4)

    print(f"\nPreview data saved to {output_file}.")

# redis에 데이터 저장
def save_to_redis(df, keywords):
    print("Starting to save data to Redis...")

    # 1. 키워드 목록 저장
    valid_keywords = [kw for kw, _ in keywords if isinstance(kw, str)]
    if not valid_keywords:
        print("No valid keywords to save.")
        return

    r.delete("keywords")  # 기존 데이터를 삭제
    r.sadd("keywords", *valid_keywords)  # 유효한 키워드만 저장
    print("Keywords saved.")

    # 2. 키워드별 기사 저장
    for keyword in valid_keywords:
        print(f"Saving articles for keyword: {keyword}")
        articles = df[df["combined"].str.contains(keyword)].copy()
        articles = articles.drop(columns=["combined"])

        r.delete(f"keyword:{keyword}:articles")  # 기존 데이터 삭제
        for _, row in articles.iterrows():
            row_data = json.dumps(row.to_dict())
            r.rpush(f"keyword:{keyword}:articles", row_data)
        print(f"Saved {len(articles)} articles for keyword: {keyword}.")

    # 3. 언론사별 기사 개수 저장
    print("Saving company stats...")
    r.delete("company:stats")
    company_counts = Counter(df["company"])
    for company, count in company_counts.items():
        r.hset("company:stats", company, count)
    print("Company stats saved.")

    # 4. 키워드별 기사 개수 저장
    print("Saving keyword stats...")
    r.delete("keyword:stats")
    for keyword in valid_keywords:
        count = len(df[df["combined"].str.contains(keyword)])
        r.hset("keyword:stats", keyword, count)
        print(f"Keyword '{keyword}' has {count} articles.")
    print("Keyword stats saved.")

    print("Data saved to Redis.")

# 실행
if __name__ == "__main__":
    from dotenv import load_dotenv
    load_dotenv()

    # redis 초기화
    reset_redis()

    # 데이터 로드 및 텍스트 결합
    combined_texts, df = load_and_combine_text(data_folder)

    # 상위 50개 키워드 추출
    top_keywords = extract_top_keywords(combined_texts, top_n=50)
    
    # 상위 키워드 출력
    print("\n상위 50개 키워드:")
    for rank, (word, count) in enumerate(top_keywords, start=1):
        print(f"{rank}. {word} - {count}회 등장")

    # Redis에 저장
    save_to_redis(df, top_keywords)

Initializing Redis database...
Redis database cleared.


FileNotFoundError: [Errno 2] No such file or directory: '../Completed_csv'